In [0]:
import ipyparallel
import os, time
%config InlineBackend.figure_format = 'retina'
from ipyparallel import Client

In [0]:
bam_dir = "/home/lindb/wbp/concatenated/raw_mapping_samfiles/"
snp_dir = "/home/lindb/wbp/concatenated/snps/"
if not os.path.exists(snp_dir):
    os.makedirs(snp_dir)
    

In [0]:
bam_dir

In [0]:
bam_files = os.listdir(bam_dir)
bam_files = [os.path.join(bam_dir,f) for f in bam_files if f.endswith('rg.bam')]
len(bam_files),bam_files[0]

In [0]:
samtools = "/home/lindb/g/src/samtools-1.3/samtools"
bcftools = "/home/lindb/g/src/bcftools-1.3/bcftools"
assembly = '/home/lindb/wbp/concatenated/velvet/auto_data_45/contigs.fa'

In [0]:
os.chdir('/gpfs_fs/home/lindb/wbp/concatenated/velvet/auto_data_45')

In [0]:
!$samtools faidx {assembly}

In [0]:
os.path.basename(bam_files[0]).split(".fastq")[0]

In [0]:
def create_ploidy_file(args):
    import os
    bam_files, snp_dir = args
    ploidy_file = os.path.join(snp_dir, "all.ploidy")
    if not os.path.exists(ploidy_file):
        with open(ploidy_file, "w") as o:
            for b in bam_files:
                name = "%s" % os.path.basename(b).split(".fastq")[0]
                ploidy = 2
                o.write("%s\t%d\n" % (name, ploidy))
    return ploidy_file

In [0]:
rc = Client(profile="xmn")
dview = rc[:]
lview = rc.load_balanced_view()
len(dview)

In [0]:
dview['create_ploidy_file'] = create_ploidy_file

In [0]:
def call_snps(args):
    import socket, os, stopwatch
    print(socket.gethostname())
    timer = stopwatch.Timer()
    
    samtools, reference, bam_sorted, bcftools, raw_vcf, out_dir, contig_names = args 
    
    raw_vcf = os.path.join(out_dir, os.path.basename(raw_vcf))
    ploidy_file = create_ploidy_file((bam_sorted, out_dir))
    pileup = "%s mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR %s -Iugf %s %s | %s call -f GP,GQ -S %s -vmO z -o %s" % (samtools, 
                                                                     contig_names, reference, 
                                                                     ' '.join(bam_sorted), 
                                                                     bcftools,
                                                                     ploidy_file,                                                                
                                                                     raw_vcf) 
    timer.stop()
    return pileup, timer.elapsed

In [0]:
bam_files[0]

In [0]:
args = [samtools,               #samtools
        assembly,               #reference
        bam_files,              #bam_sorted
        bcftools,               #bcftools
        "samtools_1.3.vcf.gz",  #raw_vcf
        snp_dir,                #out_dir
        ""]                     #contig_names

In [0]:
pileup, elapsed = call_snps(args)

In [0]:
elapsed

In [0]:
with open(os.path.join(snp_dir, "samtools.sh"), "w") as o:
    text = '''#!/bin/bash
#$ -N samtools
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/concatenated/snps/
'''
    o.write("%s\n" % text)
    o.write("%s\n" % pileup)
#qsub it

    cd /home/lindb/wbp/concatenated/snps/
    qsub samtools.sh